<a href="https://colab.research.google.com/github/rajabia/LLM_codes/blob/main/Reading_Wikipedia_to_Answer_Open_Domain_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://github.com/kushalj001/pytorch-question-answering/blob/master/1.%20DrQA.ipynb

In [10]:
import pandas as pd
import numpy as np
import torchtext
import torch
from torch import nn
import json, re, unicodedata, string, typing, time
import torch.nn.functional as F
import spacy
from collections import Counter
import pickle
from nltk import word_tokenize
# nlp = spacy.load('en')
# from preprocess import *
# %load_ext autoreload
# %autoreload 2

In [11]:
from google.colab import drive
drive.mount('/content/drive')

!ls
!ls /content/drive/MyDrive/GoogleColab/QAWiki

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data
'Arezoo Rajabi-local community detection-Chaos paper.rar'   qanetc2id.pickle   qanetw2id.pickle
 preprocess.py						    qanettrain.pkl     squad_dev.json
 __pycache__						    qanetvalid.pkl     squad_train.json


In [ ]:
nlp = spacy.load("en_core_web_sm")
import sys
sys.path.append('/content/drive/MyDrive/GoogleColab/QAWiki')
from preprocess import *
train_data = load_json('/content/drive/MyDrive/GoogleColab/QAWiki/squad_train.json')
valid_data = load_json('/content/drive/MyDrive/GoogleColab/QAWiki/squad_dev.json')

Length of data:  442
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  University_of_Notre_Dame
Length of data:  48
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Super_Bowl_50


In [ ]:
train_list = parse_data(train_data)
valid_list = parse_data(valid_data)

In [ ]:
print('Train list len: ',len(train_list))
print('Valid list len: ',len(valid_list))
print('Train list: ',train_list[0])
print('Valid list: ',valid_list[0])
print(train_list[0].keys())

Train list len:  87599
Valid list len:  34726
Train list:  {'id': '5733be284776f41900661182', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'label': [515, 541], 'answer': 'Saint Bernadette Soubirous'}
Valid list:  {'id': '56be4db0acb8001400a502ec',

In [ ]:
train_df = pd.DataFrame(train_list)
valid_df = pd.DataFrame(valid_list)
print(train_df.head(2))
# get indices of outliers and drop them from the dataframe

drop_ids_train = filter_large_examples(train_df)
train_df.drop(list(drop_ids_train), inplace=True)

drop_ids_valid = filter_large_examples(valid_df)
valid_df.drop(list(drop_ids_valid), inplace=True)

                         id  \
0  5733be284776f41900661182   
1  5733be284776f4190066117f   

                                             context  \
0  Architecturally, the school has a Catholic cha...   
1  Architecturally, the school has a Catholic cha...   

                                            question       label  \
0  To whom did the Virgin Mary allegedly appear i...  [515, 541]   
1  What is in front of the Notre Dame Main Building?  [188, 213]   

                       answer  
0  Saint Bernadette Soubirous  
1   a copper statue of Christ  


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
vocab_text = gather_text_for_vocab([train_df, valid_df])
print("Number of sentences in the dataset: ", len(vocab_text))
%time word2idx, idx2word, word_vocab = build_word_vocab(vocab_text)
print("----------------------------------")
%time char2idx, char_vocab = build_char_vocab(vocab_text)

%time train_df['context_ids'] = train_df.context.apply(context_to_ids, word2idx=word2idx)
%time valid_df['context_ids'] = valid_df.context.apply(context_to_ids, word2idx=word2idx)
%time train_df['question_ids'] = train_df.question.apply(question_to_ids, word2idx=word2idx)
%time valid_df['question_ids'] = valid_df.question.apply(question_to_ids, word2idx=word2idx)

Number of sentences in the dataset:  118427


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


raw-vocab: 109850
vocab-length: 109852
word2idx-length: 109852
CPU times: user 8min 30s, sys: 2.22 s, total: 8min 32s
Wall time: 8min 36s
----------------------------------
raw-char-vocab: 1401
char-vocab-intersect: 232
char2idx-length: 234
CPU times: user 2.4 s, sys: 89.9 ms, total: 2.49 s
Wall time: 2.49 s
CPU times: user 14min 25s, sys: 2.84 s, total: 14min 28s
Wall time: 14min 32s


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


CPU times: user 5min 44s, sys: 1.15 s, total: 5min 45s
Wall time: 5min 47s


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


CPU times: user 4min 36s, sys: 1.21 s, total: 4min 37s
Wall time: 4min 38s


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


CPU times: user 1min 49s, sys: 448 ms, total: 1min 49s
Wall time: 1min 50s


In [ ]:
# get indices with tokenization errors and drop those indices

train_err = get_error_indices(train_df, idx2word)
valid_err = get_error_indices(valid_df, idx2word)

train_df.drop(train_err, inplace=True)
valid_df.drop(valid_err, inplace=True)
print(len(train_df), len(valid_df))

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


Number of error indices: 921
Number of error indices: 359
86403 34080


In [ ]:
# get start and end positions of answers from the context
# this is basically the label for training QA models
train_label_idx = train_df.apply(index_answer, axis=1, idx2word=idx2word)
valid_label_idx = valid_df.apply(index_answer, axis=1, idx2word=idx2word)

train_df['label_idx'] = train_label_idx
valid_df['label_idx'] = valid_label_idx

print(train_df.head(2))
folder='/content/drive/MyDrive/GoogleColab/QAWiki/'
train_df.to_pickle('qanettrain.pkl')
valid_df.to_pickle('qanetvalid.pkl')

import pickle
with open(folder+'qanetw2id.pickle','wb') as handle:
    pickle.dump(word2idx, handle)

with open(folder+'qanetc2id.pickle','wb') as handle:
    pickle.dump(char2idx, handle)


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
folder='/content/drive/MyDrive/GoogleColab/QAWiki/'
train_df = pd.read_pickle(folder+'qanettrain.pkl')
valid_df = pd.read_pickle(folder+'qanetvalid.pkl')

word2idx = pd.read_pickle(folder+'qanetw2id.pickle')
char2idx = pd.read_pickle(folder+'qanetc2id.pickle')


idx2word = {v:k for k,v in word2idx.items()}
print(idx2word[103])

part


In [ ]:
class SquadDataset:
    '''
    - Creates batches dynamically by padding to the length of largest example
      in a given batch.
    - Calulates character vectors for contexts and question.
    - Returns tensors for training.
    '''
    def __init__(self, data, batch_size):
        '''
        data: dataframe
        batch_size: int
        '''
        self.batch_size = batch_size
        data = [data[i:i+self.batch_size] for i in range(0, len(data), self.batch_size)]
        self.data = data


    def __len__(self):
        return len(self.data)

    def make_char_vector(self, max_sent_len, sentence, max_word_len=16):

        char_vec = torch.zeros(max_sent_len, max_word_len).type(torch.LongTensor)

        for i, word in enumerate(nlp(sentence, disable=['parser','tagger','ner'])):
            for j, ch in enumerate(word.text):
                if j == max_word_len:
                    break
                char_vec[i][j] = char2idx.get(ch, 0)

        return char_vec

    def get_span(self, text):

        text = nlp(text, disable=['parser','tagger','ner'])
        span = [(w.idx, w.idx+len(w.text)) for w in text]

        return span


    def __iter__(self):
        '''
        Creates batches of data and yields them.

        Each yield comprises of:
        :padded_context: padded tensor of contexts for each batch
        :padded_question: padded tensor of questions for each batch
        :char_ctx & ques_ctx: character-level ids for context and question
        :label: start and end index wrt context_ids
        :context_text,answer_text: used while validation to calculate metrics
        :ids: question_ids for evaluation
        '''

        for batch in self.data:

            spans = []
            ctx_text = []
            answer_text = []


            for ctx in batch.context:
                ctx_text.append(ctx)
                spans.append(self.get_span(ctx))

            for ans in batch.answer:
                answer_text.append(ans)

            max_context_len = max([len(ctx) for ctx in batch.context_ids])
            padded_context = torch.LongTensor(len(batch), max_context_len).fill_(1)

            for i, ctx in enumerate(batch.context_ids):
                padded_context[i, :len(ctx)] = torch.LongTensor(ctx)

            max_word_ctx = 16

            char_ctx = torch.zeros(len(batch), max_context_len, max_word_ctx).type(torch.LongTensor)
            for i, context in enumerate(batch.context):
                char_ctx[i] = self.make_char_vector(max_context_len, context)

            max_question_len = max([len(ques) for ques in batch.question_ids])
            padded_question = torch.LongTensor(len(batch), max_question_len).fill_(1)

            for i, ques in enumerate(batch.question_ids):
                padded_question[i, :len(ques)] = torch.LongTensor(ques)

            max_word_ques = 16

            char_ques = torch.zeros(len(batch), max_question_len, max_word_ques).type(torch.LongTensor)
            for i, question in enumerate(batch.question):
                char_ques[i] = self.make_char_vector(max_question_len, question)


            label = torch.LongTensor(list(batch.label_idx))
            ids = list(batch.id)

            yield (padded_context, padded_question, char_ctx, char_ques, label, ctx_text, answer_text, ids)

train_dataset = SquadDataset(train_df,16)
valid_dataset = SquadDataset(valid_df,16)

In [ ]:
a = next(iter(train_dataset))
for i in range(len(a)):
    try:
        print(a[i].shape)
    except AttributeError:
        print(len(a[i]))

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


torch.Size([16, 253])
torch.Size([16, 16])
torch.Size([16, 253, 16])
torch.Size([16, 16, 16])
torch.Size([16, 2])
16
16
16


In [ ]:
def get_glove_dict():
    '''
    Parses the glove word vectors text file and returns a dictionary with the words as
    keys and their respective pretrained word vectors as values.

    '''
    glove_dict = {}
    with open(folder+"glove.840B.300d.txt", "r", encoding="utf-8") as f:
        for line in f:
            values = line.split(' ')
            word = values[0]
            vector = np.asarray(values[1:], dtype="float32")
            glove_dict[word] = vector

    f.close()

    return glove_dict